## Note : 
Mean free path : $\lambda = \frac{1}{n \sigma}$

If $n \rightarrow 100 \times n$, then :
- $\lambda \rightarrow \frac{\lambda}{100}$
- $\text{cell size} \rightarrow \frac{\text{cell size}}{100}$ ($100^2$ more cells because the grid is 2D)
- $dt \rightarrow \frac{dt}{100}$

But you can lower, down to a certain extent, the number of particles per cell to "not incease too much the computation time".

## Representative physical length scale

The system has two lenghts : 
* lenght of the tube : $L = 0.01$ $m$
* width : $w = 0.001$ $m$

The width is the representative physical length scale that constrains the flow.

So : $K_d = \frac{\lambda}{w}$.




# ISP, thrust etc.

In [222]:
def isp(ve):
    return ve/9.81

def thrust(mass, speeds, dt):
    return mass*np.sum(speeds)/dt

In [1]:

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [1]:
%matplotlib widget

# notebook
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy as scipy
from pathlib import Path
import seaborn as sns

from lppydsmc.data.saver import Saver
from plotting import analysis
from lppydsmc.utils import physics

# plt.figure.max_open_warning = False

## Loading results

In [2]:
dir_path = Path('results/')
dz = 0.001

In [3]:
dt = 1e-8
mr = 1
name = 'neutralization_three_grids.h5'
lenght = 0.011 # m
width = 0.005
cell_size = 0.001
temperature = 23e3 # K
nb_cells = 41 # 33
volume = cell_size*cell_size*dz*nb_cells

nb_injected_particles = 10000

In [4]:
store = pd.HDFStore(dir_path/name)

In [5]:
store.keys()

['/collisions_with_walls',
 '/df',
 '/df_collision_background_gas',
 '/df_collision_with_walls',
 '/df_out_particles',
 '/mean_proba_gas',
 '/mean_proba_walls',
 '/total_deleted']

In [6]:
# defining each variable
df = store['df']
df_out_particles = store['df_out_particles']
df_collision_background_gas = store['df_collision_background_gas']
df_collision_with_walls = store['df_collision_with_walls']
total_deleted = store['total_deleted']
collisions_with_walls = store['collisions_with_walls']
mean_proba_walls = store['mean_proba_walls']
mean_proba_gas = store['mean_proba_gas']

In [7]:
choice = 'I'
gamma = 5/3. # roughly for atomes, for diatomic molecule : 7/5.


mass = physics.get_mass_part(53+1, 53, 74) # I
molecular_mass = 0.1269 # kg/mol

verbose=False

In [8]:
sound_vel = physics.speed_of_sound(molecular_mass, temperature, gamma)
print(f'Speed of sound for [{choice}] at temperature {temperature} K : {sound_vel} m/s')

Speed of sound for [I] at temperature 23000.0 K : 1584.7563179678798 m/s


## Note : 
To save a figure, you can use : `analysis.save_fig(fig, path, title = None, dpi = 400, figsize = None)`, which allows you to easily save a figure while giving it a title and changing its size.
You can also use the interactive widget.

## Choosing the part of the system to plot 

In [9]:

res_x_0, res_y_0 = 11, 5

total_nb_cells = res_x_0*res_y_0

In [10]:
fig, ax = plt.subplots(1, constrained_layout = True, figsize = (res_x_0, res_y_0));

fact = 10
volume_cell_plot = volume/(fact*fact*res_x_0*res_y_0)
duration = df.index.unique().values.shape[0]
analysis.hist2d(ax, df, bins = (fact*res_x_0,fact*res_y_0), weights = mr*np.ones(df.shape[0])/(duration*volume_cell_plot), stat = 'count'); # TODO: add cmap

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Choosing frames to plot

In [11]:
unique_index = df.index.unique().values
nb_save = unique_index.shape[0]
iterations = np.max(unique_index)
adding_period = unique_index[1]-unique_index[0] # adding period - required to
# generally speaking, you choose frames so you have the steady state
frames = unique_index[int(0.0*nb_save):nb_save] 

if(verbose):
    print(f'Available frames :  {unique_index}')
    print(f'Max iteration : {iterations}')
    print(f'Choosen frames (for plotting) : \n{frames}')

## Proba evolution

In [12]:
fig, ax = plt.subplots(2)
# ax.set_yscale('log')
ax[0].plot(mean_proba_gas.index*dt, mean_proba_gas.values, label = 'gas')
ax[1].plot(mean_proba_walls.index*dt, mean_proba_walls.values, label = 'walls');
ax[0].legend();
ax[1].legend();

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Out particles analysis

In [13]:
df_out_particles.head()

,x,y,vx,vy,vz,type
125,0.020101,0.002473,20563.186640,1294.760448,537.412261,1.0
125,0.020138,0.002488,20551.768006,1660.378031,1812.953222,1.0
126,0.020118,0.002502,20497.690301,-1987.745745,978.788179,1.0
126,0.020195,0.002276,20511.435264,640.466067,60.388648,1.0
126,0.020129,0.003092,21576.989096,886.679168,-966.083596,1.0


In [14]:
print(df_out_particles.shape[0])

8746


In [15]:
def convert_to_species(row):
    if(row['type'] == 0):
        return 'I-'
    else :
        return 'I'

In [16]:
df_out_particles['species'] = df_out_particles.apply(convert_to_species, axis = 1)

In [17]:
df_out_particles['v'] = np.sqrt(df_out_particles['vx']**2+df_out_particles['vy']**2+df_out_particles['vz']**2)
df_out_particles['energy'] = df_out_particles['v']**2
df_out_particles['angle'] = np.arctan(df_out_particles['vy']/df_out_particles['vx'])

In [18]:
ions_out = df_out_particles.loc[df_out_particles['type'] == 0]
neutral_out =  df_out_particles.loc[df_out_particles['type'] == 1]

In [19]:
results = df_out_particles[['type', 'x']].groupby('type').count();
print('Out particles neutralization percentage : {:.3} %'.format(100*results.iloc[1,0]/(results.iloc[1,0]+results.iloc[0,0])))

Out particles neutralization percentage : 99.0 %


In [20]:
vel_results = df_out_particles[['species', 'vx', 'vy', 'vz', 'v','energy']].groupby('species').agg('mean');
vel_results

,vx,vy,vz,v,energy
species,,,,,
I,13640.917390,-31.985067,-15.300679,14123.159969,2.436893e+08
I-,1575.188432,89.263011,-156.692030,2065.558824,4.875298e+06


In [21]:
fig, ax = plt.subplots()
sns.despine(fig)

sns.histplot(
    df_out_particles,
    x="vx", hue="species",
    bins = 100,
    multiple="stack",
    palette="light:m_r",
    edgecolor=".3",
    linewidth=.5,
    log_scale=False,
);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [22]:
fig, ax = plt.subplots()
sns.despine(fig)

sns.histplot(
    df_out_particles,
    # bins = 100,
    x="angle", hue="species",
    multiple="stack",
    palette="light:m_r",
    edgecolor=".3",
    linewidth=.5,
    log_scale=False,
);
ax.set_xticks([-np.pi/4, -np.pi/2, 0, np.pi/2, np.pi/4]);
# ax.set_xticklabels(['$-\frac{pi}{4}$', '$-\frac{pi}{8}$', '0', '$\frac{pi}{8}$', '$\frac{pi}{4}$']);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Thrust due to ions

In [23]:
print('Mean speed out : {:.3e} m/s.'.format(df_out_particles.sum()['vx']/nb_injected_particles))

Mean speed out : 1.182e+04 m/s.


In [223]:
debit = 2e14
delta_time =  1/debit # sec
mean_speed = df_out_particles.sum()['vx']/nb_injected_particles

print('Thrust : {:.3e} N'.format(mean_speed*mass/delta_time))

Thrust : 5.022e-07 N


In [224]:
print('Isp : {:.3e} s-1'.format(mean_speed/9.81))

Isp : 1.205e+03 s-1


## df_collision_background_gas

In [24]:
df_collision_background_gas['species'] = df_collision_background_gas.apply(convert_to_species, axis = 1)

In [25]:
print('Number of collisions with background gas : {}'.format(df_collision_background_gas.shape[0]))
df_collision_background_gas.head()

Number of collisions with background gas : 5575


,x,y,type,species
1,0.000054,0.003384,0.0,I-
1,0.000029,0.004883,0.0,I-
1,0.000014,0.004905,0.0,I-
2,0.000073,0.002246,0.0,I-
3,0.000075,0.001645,0.0,I-


In [26]:
# Ion plot
sns.set_theme(style="ticks")
sns.jointplot(data = df_collision_background_gas.loc[df_collision_background_gas['type']==0], x='x', y='y', kind="hex", color="#4CB391");

# Neutral
# sns.set_theme(style="ticks")
# sns.jointplot(data = df_collision_background_gas.loc[df_collision_background_gas['type']==1], x='x', y='y', kind="hex", color="#4CB391");

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [27]:
# both
# sns.set_theme(style="ticks")
# sns.jointplot(data = df_collision_background_gas, x='x', y='y', hue="type", s = 10) # , kind="hist") #, kind="hist")

## df_collision_with_walls

In [28]:
df_collision_with_walls['species'] = df_collision_with_walls.apply(convert_to_species, axis = 1)

In [29]:
print('Number of collisions with walls : {}'.format(df_collision_with_walls.shape[0]))
df_collision_with_walls.head()

Number of collisions with walls : 65916


,x,y,type,angle,species
1,0.000024,2.997290e-07,0.0,0.738741,I-
1,0.000012,2.315133e-05,0.0,0.378487,I-
1,0.000046,4.992539e-03,0.0,1.126633,I-
1,0.000034,4.988920e-03,0.0,0.776821,I-
1,0.000045,4.997367e-03,0.0,1.155873,I-


In [30]:
ions = df_collision_with_walls.loc[df_collision_with_walls['type']==0]
fig, ax = plt.subplots()
ax.axis('equal')
scatter = ax.scatter(ions['x'], ions['y'], s = 0.005, c = ions['type']);
# produce a legend with the unique colors from the scatter
legend1 = ax.legend(*scatter.legend_elements(),
                    loc="upper right", title="Species")
ax.add_artist(legend1);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [31]:
sns.set_theme(style="ticks")
sns.jointplot(data = ions, x='x', y='y', kind="hex", color="#4CB391");

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [32]:
fig, ax = plt.subplots()
sns.despine(fig)

sns.histplot(
    df_collision_with_walls,
    # bins = 100,
    x="angle", hue="species",
    multiple="stack",
    palette="light:m_r",
    edgecolor=".3",
    linewidth=.5,
    log_scale=False,
);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [33]:
# Neutral plot
# sns.set_theme(style="ticks")
# sns.jointplot(data = df_collision_with_walls, x='x', y='y', hue="type", s = 10) # , kind="hist") #, kind="hist")

## Particles analysis
### Plotting number of particles evolution

In [34]:
df['species'] = df.apply(convert_to_species, axis = 1)

In [195]:
df.head()

,x,y,vx,vy,vz,type,species,x_int,y_int,i,j
1,0.000008,0.004777,671.253250,472.393238,745.819121,0.0,I-,0.0,0.0046,0,23
1,0.000042,0.000270,2954.596819,1108.321428,1428.512151,0.0,I-,0.0,0.0002,0,1
1,0.000015,0.003250,1275.003919,-52.714019,-75.409367,0.0,I-,0.0,0.0032,0,16
1,0.000015,0.000041,1188.067397,548.589815,1630.475805,0.0,I-,0.0,0.0000,0,0
1,0.000002,0.003980,198.207350,-294.642191,124.374629,0.0,I-,0.0,0.0038,0,19


In [196]:
nb_parts = df['x'].groupby(df.index).agg('count').values

In [197]:
fig, ax = plt.subplots()
ax.plot(df.index.unique()*dt, nb_parts)
analysis.set_axis(ax, x = 'time', y = 'quantity')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [198]:
df_it = df.loc[df.index == 1000]
fig, ax = plt.subplots()
sns.despine(fig)

sns.scatterplot(
    data = df_it,
    # bins = 100,
    x="x", y ="y", hue="species", s = 5.
);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [199]:
df_it = df.loc[df.index == 1000]
fig, ax = plt.subplots()
sns.despine(fig)

sns.scatterplot(
    data = df_it,
    # bins = 100,
    x="x", y ="y", hue="vx", s = 10.
);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [236]:
dp = 1e-4
df['i'] = (df['x']/dp).astype(int)
df['j'] = (df['y']/dp).astype(int)

#### we could select only the I- - anyway

In [237]:
groups = df.loc[df['species'] == 'I-'].groupby(['i','j']).mean()

In [238]:
groups = groups.reset_index()

In [239]:
groups

,i,j,x,y,vx,vy,vz,type,x_int,y_int
0,0,0,0.000048,0.000051,1618.870419,-162.204307,-5.988500,0.0,0.0000,0.0000
1,0,1,0.000053,0.000150,1604.782133,60.986208,-60.733695,0.0,0.0000,0.0000
2,0,2,0.000051,0.000251,1632.221946,-54.305698,45.790731,0.0,0.0000,0.0002
3,0,3,0.000053,0.000350,1661.038475,19.040131,-46.495850,0.0,0.0000,0.0002
4,0,4,0.000052,0.000451,1673.802439,-22.751111,-53.469233,0.0,0.0000,0.0004
...,...,...,...,...,...,...,...,...,...,...
4033,199,37,0.019950,0.003773,1755.081118,-648.653102,2012.437490,0.0,0.0198,0.0036
4034,199,39,0.019948,0.003961,1537.845303,56.815961,-1069.583467,0.0,0.0198,0.0038
4035,199,40,0.019948,0.004034,1722.266042,468.472150,-272.613705,0.0,0.0198,0.0040
4036,199,43,0.019936,0.004370,2353.611508,1858.771535,629.298467,0.0,0.0198,0.0042


In [240]:
groups['i'].max()

199

In [241]:
fig, ax = plt.subplots()

values = {
    'width' : 3*dp,
    'headwidth' : 3,
    'headlength' : 5
}

ax.quiver(dp*groups['i'], dp*groups['j'], groups['vx'], groups['vy'], groups['vx'], **values);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [242]:
U,V = np.zeros((groups['j'].max()-groups['j'].min(),groups['i'].max()-groups['i'].min()), dtype = float), np.zeros((groups['j'].max()-groups['j'].min(),groups['i'].max()-groups['i'].min()), dtype = float)
X, Y = np.arange(groups['i'].min(), groups['i'].max()),  np.arange(groups['j'].min(), groups['j'].max())

for x in X:
    for y in Y:
        row = groups.loc[(groups['i'] == x) &  (groups['j'] == y)]
        try :
            U[y,x] = row['vx']
            V[y,x] = row['vy']
        except ValueError as e:
            pass

In [243]:
print(U.shape, V.shape)
print(X.shape, Y.shape)
speed = np.sqrt(U**2+V**2)
lw = 2 * speed / (speed.max()+1e-8)

(49, 199) (49, 199)
(199,) (49,)


In [245]:
fig, ax = plt.subplots()
# ax.axis('equal')
ax.streamplot(x=dp*X,y=dp*Y,u=U,v=V, density = 3, color=U, linewidth=lw, cmap='plasma'); 

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …